In [28]:
# from google.colab import files
# uploaded = files.upload()

In [18]:
import os
font_dir='/home/DataAugmentation/font/TTF'
fonts=[]
for font in os.listdir(font_dir):
    fonts.append(os.path.join(font_dir,font))

In [19]:
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
fonts[0]
print len(fonts)

71


In [20]:
from PIL import Image  
from PIL import ImageDraw  
from PIL import ImageFont  
import textwrap
import os  
from random import randint  
import textwrap
import shutil
import numpy as np

In [21]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#   process = psutil.Process(os.getpid())
#   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
#   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [22]:
import tensorflow

In [23]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense, Dropout
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from PIL import Image
import os
import cv2


def LeNetbuild(width, height, depth, classes, weightsPath=None):
    # initialize the model
    model = Sequential()
        # first set of CONV => RELU => POOL
    model.add(Convolution2D(20, 3, 3, border_mode="same",
        input_shape=(height, width, depth)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        # second set of CONV => RELU => POOL
    model.add(Convolution2D(50, 5, 5, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))
        # set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))
        # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weightsPath is not None:
        model.load_weights(weightsPath)

    # return the constructed network architecture
    return model

In [24]:
from PIL import Image
import cv2
import numpy as np


def connectedcomponents(img):
    NL = 0;LABELS = 1;STATS = 2;CENTROIDS= 3
    _, binimg = cv2.threshold(img, 192, 255, cv2.THRESH_BINARY)
    invbinimg = cv2.bitwise_not(binimg)
    cc = cv2.connectedComponentsWithStats(invbinimg, connectivity=8, ltype=cv2.CV_32S)

    candidates = []
    for i in range(1, cc[NL]):
        l,t,w,h,a = cc[STATS][i]
        candidates.append((i,(l,t,l+w,t+h)))

    for i in range(len(candidates)):
        start_x=candidates[i][1][0]
        start_y=candidates[i][1][1]
        end_x=candidates[i][1][2]
        end_y=candidates[i][1][3]
        character=img[start_y:end_y,start_x:end_x]
        height,width=character.shape[0],character.shape[1] 
        nheight=18
        nwidth=int(round(width*(18.0/height)))
        msize=max(nheight+4,nwidth+2)
        image=Image.fromarray(character)
        image=image.resize((nwidth,nheight))
        char_image = Image.new('L',(msize,msize),255)
        offset = (1,4)
        char_image.paste(image, offset)
        return char_image


In [25]:
import numpy as np
import os
import cv2

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col= image.shape
        mean = 0
        sigma = 24
        gauss = np.random.normal(mean,sigma,(row,col))
        ln = image + gauss/10
        mn = image + gauss/3
        hn = image + gauss
#         np.clip(ln, 0,255,out=ln)
#         np.clip(mn, 0,255,out=mn)
#         np.clip(hn, 0,255,out=hn)
        return [ln,mn,hn]
    elif noise_typ == "s&p":
        row,col= image.shape
        salt = np.copy(image)
        num_salt = np.random.randint(1,8)
        coords = [np.random.randint(1, i-1, int(num_salt)) for i in image.shape[:2]]
        hns = num_salt/2
        if hns > 0:
            modcoords = [coord[:hns] for coord in coords]
            modcoords[0] += np.random.randint(-1,2, hns)
            modcoords[1] += np.random.randint(-1,2, hns)
            salt[modcoords] = 255
        salt[coords] = 255
        pepper = np.copy(image)
        num_pepper = np.random.randint(1,8)
        coords = [np.random.randint(1, i-1, int(num_pepper)) for i in image.shape[:2]]
        hns = num_pepper/2
        if hns > 0:
            modcoords = [coord[:hns] for coord in coords]
            modcoords[0] += np.random.randint(-1,2, hns)
            modcoords[1] += np.random.randint(-1,2, hns)
            pepper[modcoords] = 128
        pepper[coords] = 128
        return [salt, pepper]
    elif noise_typ =="speckle":
        row,col = image.shape
        gauss = np.random.randn(row,col)
        ln = image + image * gauss/6
        mn=image + image * gauss/4
        #     high_noisy=image + image * gauss/2
        np.clip(ln, 0,255,out=ln)
        np.clip(mn, 0,255,out=mn)
        return [ln,mn]
    elif noise_typ=='downscale':
        ds1=image.copy()
        ds1[ds1 < 128] += 64
        ds2=image.copy()
        ds2[ds2 < 128] += 100
        return [ds1.astype(np.uint8),ds2.astype(np.uint8)]
    elif noise_typ=="translation":
        copy=image[:,0:-2]
        b=np.full([copy.shape[0],2], 255, np.uint8)
        translated=np.append(b,copy,axis=1)
        return [translated]


In [52]:
gauss = np.random.normal(0,24,(4,4))
print gauss

[[ 40.34541078  15.33579919 -14.98227031 -25.50322889]
 [-31.0142285  -11.48817302 -15.26672506 -56.44674336]
 [ -1.48847526  -6.80019226  13.50691879   2.67967844]
 [ 13.01248654  45.4495772  -16.54998827  45.48824048]]


In [50]:
img=Image.open('/home/561_GothamLight.ttffs16.png')
image=np.asarray(img)
ds1=image.copy()
ds1[ds1 < 128] += 64
ds2=image.copy()
ds2[ds2 < 128] += 100
# ds1=ds1.astype(np.uint8)
# ds2=ds2.astype(np.uint8)
print ds1
print ds2

[[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
 [255 160 139 141 139 139 140 139 141 152 181 185 250 255 253 255]
 [255 148 117 119 117 117 116 117 120  74  94 170 249 255 252 255]
 [255 255 255 255 255 255 255 255 255 143 191 254 255 255 254 255]
 [255 255 255 255 255 255 255 255 240 189 143 255 255 253 255 255]
 [255 253 251 251 251 247 250 240 131 140 220 254 255 254 255 255]
 [255 255 255 255 255 251 255 226 136 170 255 253 255 255 255 255]
 [255 255 255 255 254 255 242 173 178 201 253 252 255 255 255 255]
 [255 255 255 255 251 254 188 153 192 250 254 254 255 255 255 255]
 [255 255 255 255 252 255 157 139 233 255 251 255 255 255 255 255]
 [255 255 253 255 255 195 188 158 245 253 251 255 255 255 255 255]
 [255 255 252 255 255 175 162 245 255 255 254 255 255 255 255 255]
 [255 253 254 253 224 166 128 255 255 252 255 255 255 255 255 255]
 [255 250 255 250 183 187 216 254 255 253 255 255 255 255 255 255]
 [255 252 251 208 131 166 255 253 255 255 255 255 255 255 255 

In [55]:
data=[]
labels=[]
noises=['s&p','gauss','speckle','downscale','translation']
def GenerateCharacters(): 
    for filename in fonts: 
        for char in characters:
            if filename.lower().find('futur')!=-1 and char=='j':
                continue;
            for font_size in font_sizes:
                font =ImageFont.truetype(filename, font_size) 
                (font_width,font_height) = font.getsize(char) 
                msize=max(font_width,font_height)
                char_image = Image.new('L',(msize,msize),background_color)  
                draw = ImageDraw.Draw(char_image) 
                if filename.lower().find('italic')!=-1:
                    offset = (2,0)
                else:
                    offset=(0,0)
                draw.text(offset,char,fillcolor,font=font)
                if char in ['.',',','-','r','u','n','e','o','a']:
                    char_image=np.asarray(char_image)
                    char_image=connectedcomponents(char_image) 
                char_image=char_image.resize((22,22))
                char_image=np.asarray(char_image)
                final_image=cv2.bitwise_not(char_image)
                newpath='/home/Desktop/Integrand/DataAugmentation/Noisy/'
                for noise in noises:
                    nimages = noisy(noise,char_image)
                    i=1
                    for nimg in nimages:
                        nimg.save(os.path.join(newpath,noise+char+str(i))
                        i+=1

SyntaxError: invalid syntax (<ipython-input-55-1c3ccc0acc01>, line 32)

In [29]:
from string import letters,digits,punctuation
import string
knownpuncs = "%$+-/&)(@.,"
digits=list(digits)
letters=list(letters)
knownpuncs=list(knownpuncs)
classes=len(digits)+len(letters)+len(knownpuncs)
print classes


73


In [30]:
fillcolor=0
background_color=255
characters=digits+letters+knownpuncs
medium_sizes = [22,23,24,25,26,27]
large_sizes = [32, 36, 40]

font_sizes=medium_sizes+large_sizes
GenerateCharacters()

In [44]:
i=55
# Image.fromarray(data[i])
print data[i]

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0   0 186 241 231 147  17   0   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0   0 110 140  19  37 203 221  27   0   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   0  70 254   0   0   0  32 251 199   2   0   0   0   0   0   0   0
    0   0   0   0]
 [  0   3 216 255   0   0   0   0 176 255 101   0   0   0   0   0   0   0
    0   0   0   0]
 [  0  67 255 251   0   0   0   0 115 255 209   0   0   0   0   0   0 

In [32]:
print labels[i]

2


In [0]:
data=np.array(data,dtype="float")/255.0
labels=np.array(labels)

In [71]:
print labels
data = data.reshape(data.shape[0], 22,22,1)
print data.shape, labels.shape

['0' '0' '0' ... ',' ',' ',']
(511335, 22, 22, 1) (511335,)


In [72]:
print data[0]

[[[0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]]

 [[0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]]

 [[0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [0.0000000e+00]
  [1.1072665e-03]
  [2.7066513e-03]
  [3.5524799e-03]
  [3.5524799e-03]
  [3.8446751e-03]
  [3.5678586e-03]
  [2.7

In [73]:
print labels[0]

0


In [0]:
from sklearn.externals import joblib

In [74]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

(trainvX, testX, trainvY, testY) = train_test_split(data, labels, test_size=0.20, random_state=23)
(trainX, valX, trainY, valY) = train_test_split(trainvX, trainvY, test_size=0.05, random_state=23)
lb = LabelBinarizer().fit(labels)
trainY = lb.transform(trainY)
testY = lb.transform(testY)
valY = lb.transform(valY)
joblib.dump(lb,'saved_encoding.pkl')

['saved_encoding.pkl']

In [0]:
# from sklearn.preprocessing import LabelBinarizer
# import os
# labels=[]
# path='/home/somit/Desktop/Integrand/DataAugmentation/Dataset/'
# for folder in os.listdir(path):
#     labels.append(folder)
# print labels
# lb=LabelBinarizer()
# labels=lb.fit_transform(labels)
# labels

In [0]:
from sklearn.externals import joblib
lb = joblib.load('saved_encoding.pkl')

In [75]:
# convert the labels from integers to vectors
#trainY = to_categorical(trainY, num_classes=2)
#testY = to_categorical(testY, num_classes=2)
EPOCHS = 2
INIT_LR = 1e-3
BS = 128

# initialize the model
print("compiling model...")
model = LeNetbuild(width=22, height=22, depth=1, classes=73)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

compiling model...


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), padding="same", input_shape=(22, 22, 1...)`
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`


In [76]:
print("[INFO] training network...")
model.fit(x = trainX, y= trainY, epochs = 2, shuffle = True, batch_size = 128, validation_data=(valX, valY),verbose=2)

[INFO] training network...
Train on 388614 samples, validate on 20454 samples
Epoch 1/2
 - 32s - loss: 4.2902 - acc: 0.0133 - val_loss: 4.2903 - val_acc: 0.0143
Epoch 2/2


KeyboardInterrupt: ignored

In [68]:
model.evaluate(testX, testY)

102267/102267 [==============================] - 11s 106us/step


[4.290277325700324, 0.013278965844309503]

In [107]:
# serialize model to JSON
model_json = model.to_json()
with open("Colab_Model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Colab_Model_Weights.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files
files.download('Colab_Model.json')
files.download('Colab_Model_Weights.h5')
files.download('saved_encoding.pkl')

In [0]:
test_data_path='/home/somit/Desktop/Integrand/31/Images'
test_data=[]
for impath in os.listdir(test_data_path):
    filename=os.path.join(test_data_path,impath)
    img=cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img=cv2.bitwise_not(img)
    img=img.reshape(16,16,1)
    test_data.append(img)
test_data=np.array(test_data,dtype="float")/255.0

In [0]:
# test_data=np.array(test_data).astype(np.float)/255.0
# np.array(img, dtype = "float") /255.0
test_data[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
      

In [0]:
# img

In [0]:
encoded_labels=model.predict(test_data,batch_size=64, verbose=1,steps=None)
final_labels=lb.inverse_transform(encoded_labels)

1139/1139 [==============================] - 0s 177us/step


In [0]:
#final_labels

In [0]:
print np.argmax(encoded_labels[0])
print np.argmax(encoded_labels[1])
print final_labels[0]
print final_labels[1]
# for i in range(len(final_labels)):
#     print (np.argmax(encoded_labels[i]),final_labels[i])
# for probabilities in encoded_labels:
#     print max(probabilities)

40
33
T
O


In [0]:
print test_data.shape
print len(final_labels)

(1139, 16, 16, 1)
1139


In [0]:
Image_predictions='/home/somit/Desktop/Integrand/31/Predictions_with_Inversion'
if not os.path.isdir(Image_predictions):
    os.makedirs(Image_predictions)
def clear_predictions():
    for files in os.listdir(Image_predictions):
        os.unlink(os.path.join(Image_predictions,files))

In [0]:
clear_predictions()
i=0
for images in os.listdir(test_data_path):
    filename=os.path.join(test_data_path,images)
    img=cv2.imread(filename,0)
    cv2.imwrite(os.path.join(Image_predictions,"Label "+str(i)+" = "+final_labels[i]+".jpg"),img)
    i+=1 

In [0]:
predictions=[]
for i in range(len(final_labels)):
    predictions.append({'character':final_labels[i],'confidence':max(encoded_labels[i])})
    
predictions[0]

{'character': 'W', 'confidence': 0.9808153}

In [0]:
from keras.models import load_model
from keras.models import model_from_json
#model = load_model('/home/vishal/model.h5')

# load json and create model
json_file = open('model1.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model1.h5")
print("Loaded model from disk")

model.save('model1_OCR.hdf5')
#loaded_model=load_model('model1_OCR.hdf5')

Loaded model from disk
